In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://www.w3schools.com/python/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
page = requests.get(url)

txt = page.text
status = page.status_code

In [3]:
#printing the staus of the requests
print(status)

200


In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
# creating a list of all common heading tags
heading_tags = ["h1","h2","h3"]
for tags in soup.find_all(heading_tags):
    print(tags.name + ' -> ' + tags.text.strip())

h2 -> Tutorials
h3 -> HTML and CSS
h3 -> Data Analytics
h3 -> XML Tutorials
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> Web Building
h3 -> Data Analytics
h3 -> XML Tutorials
h2 -> References
h3 -> HTML
h3 -> CSS
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> XML
h3 -> Character Sets
h2 -> Exercises and Quizzes
h3 -> Exercises
h3 -> Quizzes
h3 -> Courses
h3 -> Certificates
h2 -> Python Tutorial
h2 -> File Handling
h2 -> Python Modules
h2 -> Python Matplotlib
h2 -> Machine Learning
h2 -> Python MySQL
h2 -> Python MongoDB
h2 -> Python Reference
h2 -> Module Reference
h2 -> Python How To
h2 -> Python Examples
h1 -> Python Tutorial
h2 -> Learn Python
h2 -> Learning by Examples
h3 -> Example
h2 -> Python File Handling
h2 -> Python Database Handling
h2 -> Python Exercises
h2 -> Test Yourself With Exercises
h2 -> Exercise:
h2 -> Python Examples
h2 -> Python Quiz
h2 -> Python Reference
h2 -> Download Python
h2 -> Kickstart your career
h2 -> Report Error
h2 -> Thank Yo

In [6]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left"><span class="left_h2">Python</span> Tutorial</h2>,
 <h2 class="left">File Handling</h2>,
 <h2 class="left">Python Modules</h2>,
 <h2 class="left">Python Matplotlib</h2>,
 <h2 class="left">Machine Learning</h2>,
 <h2 class="left">Python MySQL</h2>,
 <h2 class="left">Python MongoDB</h2>,
 <h2 class="left">Python Reference</h2>,
 <h2 class="left">Module Reference</h2>,
 <h2 class="left">Python How To</h2>,
 <h2 class="left">Python Examples</h2>]

In [7]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

Python Tutorial
File Handling
Python Modules
Python Matplotlib
Machine Learning
Python MySQL
Python MongoDB
Python Reference
Module Reference
Python How To
Python Examples


In [8]:
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

In [9]:
# getting the Headings, Sub-Headings, Links and Contents

# CREATING RELEVANT VARIABLES
tag = ''
page_table = []
course_heading = []
course_subheading = []
course_links = []
sub_page_content_list = []
unordered_lists = []
ordered_lists = []
list_headings = []
examples_per_page = []
excercises = []
base_url = "https://www.w3schools.com/python/"
course_url = "https://www.w3schools.com/python/"

# FINDING ALL THE LINKS PRESENT IN THE LEFT MENU AND ITERATING OVER THE LINKS TO SCRAPE THE DATA
test = soup.find('div', id='leftmenuinnerinner')

for i in test:
    if 'h2' in str(i):
        tag = i.text
    else:
        if (tag!='') and ('a href' in str(i)) and ('div class="tut_overview"' not in str(i)):
            course_heading.append(tag)
            course_subheading.append(i.text)
            extra = i.get('href')
            if extra[0] == '/':
                base_url = "https://www.w3schools.com"
            
            page_url = base_url + extra
            course_links.append(page_url)
            sub_page = requests.get(page_url)
            soup_page = BeautifulSoup(sub_page.content, 'html.parser')
            
            page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
            page_examples = soup_page.find_all('div', class_ = 'w3-code notranslate pythonHigh')
            
            ##################################################################################################
            
            # Examples per page
            pg_example = []
            for example in page_examples:
                nw = []
                for i in list(example):
                    if str(i) == '<br/>':
                        pass
                    else:
                        nw.append(i)
                pg_example.append(nw)
            examples_per_page.append(pg_example)
                
            ##################################################################################################
            
            # Search for tables 
            tables = soup_page.find_all('table')

            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                for row in all_rows:
                    row_list = row.find_all('td')
                    dataRow = []
                    for cells in row_list:
                        dataRow.append(cells.text)
                    data.append(dataRow)

            table_new = []
            row = ''
            for i in range(1, len(data)):
                if len(data[i]) == 0:
                    row = row[:-3]
                    table_new.append(row) 
                    row = ''
                for j in data[i]:
                    row += j + "|||"
                    # using separator for each column
                row = row[:-3]
                # using '&&&' separator for each row
                row += '&&&'
                #print(row)
            row = row[:-3]
            if len(row) !=0:
                if row[0] == '&':
                    row = row[3:]
            table_new.append(row)
            table_string = ''
            for each in table_new:
                table_string += each + '$$$'
            table_new = table_string[:-3]
            page_table.append(table_new)
            
            ##################################################################################################
            
            # Points for Unordered lists
            ul = soup_page.find_all('ul')
            unorder = []
            for i in ul:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                unorder.append(start_string)
            if len(unorder) == 0:
                unorder = None
            else:
                unorder_string = ''
                for i in unorder:
                    unorder_string += i +'|||'
                unorder = unorder_string[:-3]
            unordered_lists.append(unorder)
            
                        
            ##################################################################################################
            #Points for ordered list.
            ol = soup_page.find_all('ol')
            order = []
            for i in ol:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                order.append(start_string)
            
            if len(order) == 0:
                order = None
            else:
                order_string = ''
                for i in order:
                    order_string += i +'|||'
                order = order_string[:-3]
            ordered_lists.append(order)
            
            ##################################################################################################
            
            # Headings and Subheadings and Description
            
            list_headings_per_page = []
            for page_data in page_content:
                for j in str(page_data).split('\n'):
                    list_headings_per_page.append(j)
                page_paragraphs = page_data.find_all('p')
        
            content_per_page = ''
            for i in page_paragraphs:
                content_per_page += i.text + '|||'
            content_per_page = content_per_page[:-3].replace('\n', '')
            sub_page_content_list.append(content_per_page)
            
            prev = ''
            ul_ol_heading = ''
            for i in list_headings_per_page:
                if ('ul' in i) and (('h2' in prev) or ('h3' in prev)):
                    ul_ol_heading += prev[4:-5] + '|||'
                prev = i
            list_headings.append(ul_ol_heading[:-3])
#             time.sleep(5)      

In [10]:
length = len(course_links)
length

115

In [11]:
len(course_heading), len(course_subheading), len(course_links), len(sub_page_content_list), len(list_headings), len(unordered_lists),len(ordered_lists),  len(page_table), len(examples_per_page)

(115, 115, 115, 115, 115, 115, 115, 115, 115)

In [12]:
from csv import writer

# writing the file
with open('python_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('python_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,Python Tutorial,Python HOME,https://www.w3schools.com/python/default.asp,Python is a popular programming language.|||Py...,Learn Python|||Learning by Examples,NaN,NaN,NaN,"[['\nprint(""Hello, World!"")\n']]"
1,Python Tutorial,Python Intro,https://www.w3schools.com/python/python_intro.asp,Python is a popular programming language. It w...,What is Python?|||What can Python do?|||Why Py...,"web development (server-side), @@@software dev...",NaN,NaN,"[['\nprint(""Hello, World!"")\n']]"
2,Python Tutorial,Python Get Started,https://www.w3schools.com/python/python_getsta...,Many PCs and Macs will have python already ins...,NaN,NaN,NaN,NaN,"[['\nprint(""Hello, World!"")\n']]"
3,Python Tutorial,Python Syntax,https://www.w3schools.com/python/python_syntax...,"As we learned in the previous page, Python syn...",NaN,NaN,NaN,NaN,"[['\n if 5 > 2:', '\xa0\nprint(""Five is great..."
4,Python Tutorial,Python Comments,https://www.w3schools.com/python/python_commen...,Comments can be used to explain Python code.||...,Multi Line Comments,NaN,NaN,NaN,"[['\n#This is a comment', '\nprint(""Hello, Wor..."


In [18]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

In [19]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)

In [20]:
# Making a separate column for those heading which are not associated with the Lists
new_col = [None for i in range(len(course_links))]
dataset['Other Headings'] = pd.Series(new_col)
for i in range(len(dataset['Unordered Lists'])):
    if dataset['Unordered Lists'][i] == '':
        dataset['Other Headings'][i] = dataset['List Headings'][i]
        dataset['List Headings'][i] = ''

In [21]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)

In [22]:
# viewing the top 10 rows
dataset.head(10)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples,Other Headings
0,Python Tutorial,Python HOME,https://www.w3schools.com/python/default.asp,Python is a popular programming language.|||Py...,,,,,"[['\nprint(""Hello, World!"")\n']]",
1,Python Tutorial,Python Intro,https://www.w3schools.com/python/python_intro.asp,Python is a popular programming language. It w...,What is Python?|||What can Python do?|||Why Py...,"web development (server-side), @@@software dev...",,,"[['\nprint(""Hello, World!"")\n']]",
2,Python Tutorial,Python Get Started,https://www.w3schools.com/python/python_getsta...,Many PCs and Macs will have python already ins...,,,,,"[['\nprint(""Hello, World!"")\n']]",
3,Python Tutorial,Python Syntax,https://www.w3schools.com/python/python_syntax...,"As we learned in the previous page, Python syn...",,,,,"[['\n if 5 > 2:', '\xa0\nprint(""Five is great...",
4,Python Tutorial,Python Comments,https://www.w3schools.com/python/python_commen...,Comments can be used to explain Python code.||...,,,,,"[['\n#This is a comment', '\nprint(""Hello, Wor...",
5,Python Tutorial,Python Variables,https://www.w3schools.com/python/python_variab...,Variables are containers for storing data valu...,,,,,"[['\nx = 5', '\ny = ""John""', '\nprint(x)', '\n...",
6,Python Tutorial,Python Data Types,https://www.w3schools.com/python/python_dataty...,"In programming, data type is an important conc...",,,,"Numeric Types:|||int, float,\n complex&&&Se...","[['\nx = 5', '\nprint(type(x))\n']]",
7,Python Tutorial,Python Numbers,https://www.w3schools.com/python/python_number...,There are three numeric types in Python:|||Var...,Example,int@@@float@@@complex,,,"[['\nx = 1\xa0\xa0\xa0 \n # int', 'y = 2.8\xa...",
8,Python Tutorial,Python Casting,https://www.w3schools.com/python/python_castin...,There may be times when you want to specify a ...,,int() - constructs an integer number from an i...,,,"[['\nx = int(1)\xa0\xa0 # x will be 1', '\ny =...",
9,Python Tutorial,Python Strings,https://www.w3schools.com/python/python_string...,Strings in python are surrounded by either sin...,,,,,"[['\nprint(""Hello"")', ""\nprint('Hello')\n""], [...",
